# Factorial Analysis

In [4]:
# install.packages('ggrepel')
# install.packages('devtools')  
# install_github("ropensci/plotly")
# install.packages('ggplot2')
# install.packages('gridExtra')
library(devtools)
library(plotly)
library(ggplot2)
library(ggrepel)
# library(scales)
library(gridExtra)

In [15]:
df <- read.csv('../input/data.csv')
df$X <- NULL
rownames(df) <- df$State
df$State <- NULL

## Define Functions

In [24]:
pca <- function(data){
    #Restrict to active/projected variables
    data.active <- data[,c("Age.18_29",
                           "Age.30_44",
                           "Age.45_64",
                           "Age.Over65",
                           "White",
                           "AfricanAmerican",
                           "Latino",
                           "CollegeEducation",
                           "PovertyRate",
                           "HouseholdIncomeOver100k",
                           "ObesityRate",
                           "GunOwnership",
                           "UnemploymentRate")]
    colnames(data.active) <- c("Age.18_29"
                               ,"Age.30_44"
                               ,"Age.45_64"
                               ,"Age.65+"
                               ,"White"
                               ,"AfricanAmerican"
                               ,"Latino"
                               ,"CollegeEducation"
                               ,"Poverty"
                               ,"Income.100k+"
                               ,"Obesity"
                               ,"GunOwner"
                               ,"Unemployment")

    data.proj <- data[, c("Trump.2016","Clinton.2016",
                          "Trump.PollError.2016",
                          "Obama.2012", "Romney.2012"
                          )]

    scale_factor <- 2

    #Scale the matrix
    data.scaled.indv <- scale(data.active, center = TRUE, scale = TRUE)

    #We create the correlation matrix
    data.correl.indv <- (1/length(data.active[,1]))*t(data.scaled.indv)%*%data.scaled.indv

    #Eigen-values and vectors
    data.eig.indv <- eigen(data.correl.indv)
    eigenvalues.indv <- as.data.frame(data.eig.indv$values / sum(data.eig.indv$values))
    eigenvectors.indv <- as.data.frame(data.eig.indv$vectors)
    #plot(eigenvalues.indv)

    numberDimension <- length(which(eigenvalues.indv>10^-10))
    axes <- matrix(nrow=numberDimension, ncol=1)
    for (i in 1:numberDimension){
        axes[i] <- paste("PCA Axis", i)
    }
    eigenvectors.indv <- eigenvectors.indv[,c(1:numberDimension)]

    # New dataset:
    c.alpha.indv <- as.matrix(data.scaled.indv) %*% as.matrix(eigenvectors.indv)
    colnames(c.alpha.indv) <- axes

    d.alpha <-matrix( nrow=length(data.active[1,]), ncol=numberDimension)
    for (i in 1:length(data.active[1,])){
      for (j in 1:numberDimension){
        d.alpha[i,j] <- cor(data.active[,i],c.alpha.indv[,j])
      }
    }
    rownames(d.alpha) <- colnames(data.active)
    colnames(d.alpha) <- axes
    d.alpha <- scale_factor*d.alpha

    d.alpha.proj <- matrix(nrow=length(data.proj[1,]), ncol=numberDimension)
    for (i in 1:length(data.proj[1,])){
      for (j in 1:numberDimension){
        d.alpha.proj[i,j] <- scale_factor*cor(data.proj[,i],c.alpha.indv[,j])
      }
    }
    rownames(d.alpha.proj) <- colnames(data.proj)
    colnames(d.alpha.proj) <- axes
    
    return(list(c.alpha.indv, d.alpha))
}

## USA Analysis

In [25]:
#Exclude DC
data <- df[which(rownames(df) != "DistrictofColumbia"),]

In [26]:
pca(data)

ERROR while rich displaying an object: Error in .escape_vec(vec, "html"): expected `vec` to be a vector or factor but it is a matrix

Traceback:
1. FUN(X[[i]], ...)
2. tryCatch(withCallingHandlers({
 .     rpr <- mime2repr[[mime]](obj)
 .     if (is.null(rpr)) 
 .         return(NULL)
 .     prepare_content(is.raw(rpr), rpr)
 . }, error = error_handler), error = outer_handler)
3. tryCatchList(expr, classes, parentenv, handlers)
4. tryCatchOne(expr, names, parentenv, handlers[[1L]])
5. doTryCatch(return(expr), name, parentenv, handler)
6. withCallingHandlers({
 .     rpr <- mime2repr[[mime]](obj)
 .     if (is.null(rpr)) 
 .         return(NULL)
 .     prepare_content(is.raw(rpr), rpr)
 . }, error = error_handler)
7. mime2repr[[mime]](obj)
8. repr_html.list(obj)
9. repr_list_generic(obj, "html", "\t<li>%s</li>\n", "\t<dt>$%s</dt>\n\t\t<dd>%s</dd>\n", 
 .     "<strong>$%s</strong> = %s", "<ol>\n%s</ol>\n", "<dl>\n%s</dl>\n", 
 .     numeric_item = "\t<dt>[[%s]]</dt>\n\t\t<dd>%s</dd>\n", 

1. 
| <!--/--> | PCA Axis 1 | PCA Axis 2 | PCA Axis 3 | PCA Axis 4 | PCA Axis 5 | PCA Axis 6 | PCA Axis 7 | PCA Axis 8 | PCA Axis 9 | PCA Axis 10 | PCA Axis 11 | PCA Axis 12 | PCA Axis 13 | 
|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|
| Alabama | -3.34270709   |  0.34571967   | -1.173974826  |  0.8142003250 | -0.1136458434 | -0.476076257  | -0.159706221  | -0.185602051  |  0.032187914  |  0.359805016  |  0.02659072   | -0.0074926497 |  0.0217763183 | 
| Alaska | -0.02275023   | -4.96454297   |  0.086109891  |  1.2668962283 |  2.8438511728 | -1.897387907  |  1.024704625  |  0.143108425  | -0.723193721  | -0.067000637  | -0.20187945   | -0.2582592499 | -0.0080278846 | 
| Arizona | -0.46976332   | -0.29786552   |  0.009939442  | -2.5523275134 | -0.4485451227 |  0.601488453  |  0.360205882  |  0.056365314  |  0.001203237  |  0.411855441  | -0.15109211   |  0.0215160344 | -0.0298533820 | 
| Arkansas | -3.23739599   |  0.59672257   |  0.898034902  |  0.2005539428 | -1.1011267777 | -0.549633118  | -0.100381228  | -0.742331719  |  0.031308402  |  0.096201327  |  0.14795181   |  0.2765891595 | -0.0049047516 | 
| California |  1.82730190   | -2.48305068   | -1.695311899  | -2.1181613012 |  0.2844881444 |  0.461410388  |  0.299638949  | -0.102681148  | -0.404015187  | -0.438324114  |  0.27087554   |  0.2283009785 |  0.0032519140 | 
| Colorado |  2.96963710   | -2.32550431   |  1.083580637  | -0.3979557262 | -0.0818094767 | -0.085857833  | -1.848991887  | -0.165875234  | -0.047251740  | -0.114018053  |  0.01412371   | -0.0730394093 | -0.0003723780 | 
| Connecticut |  3.35000752   |  1.45379841   | -1.422141868  | -0.1066458453 |  0.9510172162 | -0.424209651  |  0.049461122  | -0.130553132  | -0.045417617  |  0.281401268  | -0.04206831   |  0.2197092072 |  0.0017051894 | 
| Delaware |  1.17438314   |  1.82885975   | -1.557917930  |  0.5687416589 | -0.9692389598 |  1.050008888  |  0.864818021  |  0.139609587  | -0.058207845  | -0.116778531  | -0.63688995   | -0.0061789858 | -0.0025968139 | 
| Florida | -0.19652893   |  2.08684982   | -0.772760996  | -2.4345401918 | -1.4958838668 | -0.047414124  | -0.077086049  | -0.083926661  |  0.247700970  |  0.670123368  | -0.89412000   |  0.1180291431 | -0.0049915789 | 
| Georgia | -1.29903783   | -1.54149819   | -1.998664333  |  1.5199425252 | -0.2467351286 |  0.271959747  | -0.932238559  | -0.055302975  | -0.192183622  | -0.355896617  | -0.03935798   | -0.2331044920 | -0.0068996942 | 
| Hawaii |  1.85597819   | -2.62766829   | -0.829496394  | -0.6478377561 | -3.5215662065 | -2.456082220  |  1.192857641  |  0.891871306  |  0.454063781  | -0.473370993  |  0.58476366   | -0.1042225267 |  0.0017501799 | 
| Idaho | -1.27638197   | -0.47095167   |  2.432137691  | -0.6686810162 | -0.3784349774 | -0.574670339  | -0.736914707  | -0.230287373  |  0.108638221  | -0.014320595  |  0.04320916   |  0.1740482125 |  0.0023362876 | 
| Illinois |  0.44671832   | -0.52829740   | -1.203693385  |  0.2233512924 |  0.7570231623 |  0.227691815  |  0.044944061  |  0.085668653  |  0.408259239  |  0.411170583  |  0.10780101   | -0.2185674676 | -0.0017793276 | 
| Indiana | -0.92874152   |  0.27187169   |  0.891583114  |  0.5852029637 |  0.2503914742 |  0.308268117  |  0.190010329  |  0.002091271  |  0.044616218  | -0.630905454  | -0.30039081   | -0.0156188349 |  0.0099323708 | 
| Iowa | -0.33937922   |  1.05279491   |  1.551425040  |  0.1750924953 | -0.0247430320 |  0.318678425  |  0.736535484  |  0.002138987  |  0.252972406  |  0.171429805  | -0.26310816   | -0.0246459357 | -0.0235290527 | 
| Kansas | -0.13989058   | -0.10413057   |  0.830224145  |  0.3488804104 |  0.2531112000 |  0.690828533  |  0.713557326  | -0.420324016  |  0.277752345  |  0.684371018  |  0.31873608   | -0.5396579194 | -0.0034458403 | 
| Kentucky | -2.62797720   |  0.84159281   |  0.838680606  |  0.4118235520 |  0.7600543720 |  0.079830082  | -0.181984989  |  0.078722978  |  0.042896123  | -0.288775770  |  0.63900317   |  0.2667498902 |  0.0009524273 | 
| Louisiana | -3.74160421   | -1.14098364   | -1.960125900  |  1.1741242600 |  0.2870433235 | -0.026306523  | -0.137377476  |  0.003087732  | -0.127672321  |  0.161146133  |  0.10929782   |  0.2188257102 | -0.0045485909 | 
| Maine |  0.59277235   |  4.20665476   |  0.917279970  |  0.0007948439 |  0.2740462877 | -0.008759662  | -0.155834876  |  0.031134493  |  0.165483965  | -0.441690536  |  0.43773715   | -0.1594323527 | -0.0026552678 | 
| Maryland |  2.52799129   | -0.37725921   | -2.598326830  |  2.1034173060 | -0.4756004442 | -0.411762290  | -0.097275063  | -0.663000081  |  0.534396459  |  0.378033239  | -0.08408991   |  0.0849872662 | -0.0080620962 | 
| Massachusetts |  3.62958640   |  0.28476319   |  0.073438675  |  0.2699660069 | -0.3593346247 | -0.092747955  | -0.363784601  | -0.596140026  | -0.522232135  |  0.478204545  |  0.59505685   |  0.3164791231 | -0.0107768777 | 
| Michigan | -0.71239174   |  1.49929610   | -0.310430443  |  0.5114510649 |  0.1587090367 |  0.153831250  |  0.114218799  | -0.187306819  | -0.500164786  | -0.492551680  |  0.11376253   |  0.0163434220 |  0.0002575158 | 
| Minnesota |  1.86771584   |  0.30254778   |  1.031446057  |  0.7206280186 |  0.3409914883 | -0.815050277  | -0.655267196  |  0.154915183  |  0.543449465  |  0.096591549  | -0.17254399   | -0.0736858288 | -0.0265590197 | 
| Mississippi | -4.70059401   | -0.65701385   | -2.106109460  |  1.0166154908 | -0.7338679374 |  0.319086306  | -0.541492099  |  0.105702607  | -0.514325967  | -0.149967788  |  0.07229570   |  0.2298456696 | -0.0112886591 | 
| Missouri | -0.96475782   |  0.80176962   |  0.080774632  |  0.4904329501 |  0.3824185268 |  0.556447116  |  0.129237259  |  0.448212776  |  0.136456704  | -0.004870675  |  0.05284415   | -0.1641436351 | -0.0299795769 | 
| Montana | -0.03651260   |  1.28659204   |  1.690381117  | -0.9511623352 | -0.4687131379 | -1.343409308  | -1.048923169  |  0.396682163  | -0.912770873  |  0.136519175  | -0.18259767   | -0.1330041205 | -0.0070686213 | 
| Nebraska |  0.63125968   | -0.04341163   |  1.462739503  |  0.4965879743 | -0.5193858090 |  1.383806898  |  0.311223126  |  0.009545288  |  0.740883104  | -0.009954766  |  0.09122203   | -0.1112339428 |  0.0292470917 | 
| Nevada | -0.59016435   | -0.77432556   | -0.450277004  | -1.4172413851 |  0.0756662775 | -0.441019910  | -0.059842873  |  0.417089525  |  0.488131819  | -0.775702320  | -0.72869478   | -0.0329752008 |  0.0124596675 | 
| NewHampshire |  3.71826039   |  2.61357049   |  0.954797700  |  1.1177500940 |  0.3386817516 | -0.042827640  |  0.284158574  | -0.646437949  |  0.046209147  | -0.708876472  |  0.03134528   |  0.3548736507 |  0.0098248538 | 
| NewJersey |  3.26790076   |  0.81952421   | -2.150158962  | -0.0618337346 |  0.7145465307 | -0.234527696  | -0.034776726  |  0.085718574  |  0.673640801  | -0.036374235  | -0.03222917   |  0.2593234616 |  0.0021466460 | 
| NewMexico | -1.79654663   | -0.81561640   | -1.006475625  | -4.6741943324 |  0.9919684928 | -0.236627593  | -0.092934944  | -1.114338184  | -0.187942985  | -0.189616495  |  0.28396450   | -0.2581267216 |  0.0094993371 | 
| NewYork |  1.74814293   | -0.41303155   | -1.917775654  | -0.6582915571 | -0.1694383281 |  0.783612803  | -0.253488907  |  0.433362303  | -0.641569670  | -0.056539869  |  0.27528417   |  0.1299021247 | -0.0098361614 | 
| NorthCarolina | -0.91805956   |  0.32964593   | -1.071351174  |  0.6282834986 | -0.5240759932 |  0.306675516  | -0.497985932  |  0.167795694  | -0.291665705  | -0.107721101  | -0.10285918   | -0.3079523459 |  0.0199235584 | 
| NorthDakota |  0.10558196   | -2.26183546   |  2.542009554  |  0.7022992124 | -0.7098280811 |  0.264374053  |  1.304271990  | -0.875426477  | -1.100014393  |  0.538264527  | -0.07991101   |  0.3083688978 |  0.0036325500 | 
| Ohio | -0.38888048   |  1.31269314   | -0.123450206  |  0.4519556578 |  0.2873683014 |  0.625298472  | -0.009670695  |  0.626545271  | -0.076641312  | -0.644432468  | -0.18370344   |  0.1324810285 | -0.0248420454 | 
| Oklahoma | -1.82220664   | -0.45049975   |  0.207788279  |  0.0310532996 |  0.0006262537 |  0.614072841  |  0.703399944  |  0.619566300  |  0.254983871  | -0.116164530  |  0.35302057   | -0.2532257859 | -0.0038375588 | 
| Oregon | -0.06488517   |  0.89736142   |  0.385231585  | -0.7308209953 |  0.4370732440 |  0.410289101  | -0.385600495  |  0.917394723  |  0.526936488  |  0.662160998  |  0.73388249   | -0.1056868119 |  0.0163770388 | 
| Pennsylvania |  0.05270392   |  1.64950423   | -0.564755655  | -0.1946907536 |  0.7091408981 | -0.118499567  |  0.530148766  |  0.560761802  | -0.156789580  |  0.398335799  | -0.45424541   | -0.0873879957 |  0.0020307761 | 
| RhodeIsland |  1.69390230   |  0.94213972   | -0.719198168  | -0.8209787076 |  0.6808902052 |  1.159650855  |  0.478658241  |  0.822794680  | -1.132510863  | -0.002576646  |  0.05019718   |  0.1242338372 |  0.0270814519 | 
| SouthCarolina | -2.00185128   |  0.45787977   | -0.984705301  |  0.6825734737 | -1.1019945898 | -0.391681221  | -0.283218662  | -0.417085082  | -0.412243120  |  0.205009332  | -0.45351561   | -0.2018219524 |  0.0204402666 | 
| SouthDakota | -0.19507863   |  0.60158166   |  2.088436186  |  0.2420649316 | -1.1679089633 |  0.386346762  |  0.360987556  | -0.296717613  | -0.131389548  | -0.373362363  |  0.29867321   | -0.0456729619 | -0.0036615901 | 
| Tennessee | -1.95401018   |  0.57327566   | -0.013017623  |  0.7570643525 | -0.0554414315 |  0.050346102  | -0.093838455  | -0.132090481  |  0.158758297  | -0.020517648  |  0.11811218   | -0.0422371605 | -0.0026233546 | 
| Texas | -0.34439674   | -3.13871544   | -0.393570989  | -0.8456921760 |  0.6319247102 |  0.962825154  |  0.114600390  | -1.463518834  |  1.014688338  | -0.382106368  | -0.02070271   | -0.0460053758 | -0.0112707897 | 
| Utah |  1.00109395   | -5.82916612   |  2.864919487  |  0.4055283672 | -0.7427353522 |  1.691772827  | -0.668585794  |  0.895513686  |  0.055659541  |  0.130419588  | -0.35155352   |  0.1270756622 |  0.0036044556 | 
| Vermont |  2.53304233   |  2.74897314   |  1.345735738  |  0.2631313301 | -0.0785519109 | -0.455394558  | -0.290226724  | -0.473462287  | -0.829511724  | -0.069560630  | -0.13775569   | -0.5722625943 | -0.0026445690 | 
| Virginia |  1.51086935   | -0.90684282   | -0.972733317  |  1.3258333326 | -0.3082761812 | -0.149555942  | -0.214845900  | -0.393893610  |  0.310342478  |  0.500948031  |  0.09880819   | -0.0811034367 |  0.0209382471 | 
| Washington |  1.31942431   | -0.54356297   | -0.023356750  | -0.1418171134 |  0.8155706380 | -0.214123120  | -0.272161936  |  0.935385862  |  0.149213214  |  0.260087112  |  0.18449000   | -0.0003975322 | -0.0025708520 | 
| WestVirginia | -3.53950798   |  2.56953185   |  1.077876849  | -0.5913762677 |  0.9849215764 | -0.900032520  |  0.265909603  |  0.461700860  |  0.407223984  |  0.334724577  |  0.16437699   |  0.3557934910 |  0.0035448479 | 
| Wisconsin |  0.16934953   |  1.27854291   |  0.976657638  |  0.5555526038 |  0.3417891906 | -0.221979958  |  0.166143230  | -0.291102292  |  0.340584316  | -0.389005087  | -0.17551124   | -0.1708053450 |  0.0078918934 | 
| Wyoming | -0.34162156   | -0.95828322   |  1.698552256  | -0.0475447553 |  1.2435687013 | -1.062963315  | -0.045254755  |  0.174917999  |  0.559073868  |  0.104180013  | -0.52860574   |  0.3644726015 |  0.0180214495 | 



2. 
| <!--/--> | PCA Axis 1 | PCA Axis 2 | PCA Axis 3 | PCA Axis 4 | PCA Axis 5 | PCA Axis 6 | PCA Axis 7 | PCA Axis 8 | PCA Axis 9 | PCA Axis 10 | PCA Axis 11 | PCA Axis 12 | PCA Axis 13 | 
|---|---|---|---|---|---|---|---|---|---|---|---|---|
| Age.18_29 | -0.05465893   | -1.84822351   |  0.20547145   |  0.09246798   |  0.02726802   |  0.30393321   |  0.35435890   | -0.151715121  | -0.53365712   |  0.01349401   | -0.05305652   | -0.025228803  | -1.353746e-02 | 
| Age.30_44 | -0.44854284   | -1.74234590   |  0.33106033   |  0.29895663   |  0.08215199   |  0.10121837   | -0.43825423   |  0.272809800  |  0.52576448   | -0.03404433   |  0.05063367   |  0.017042371  | -1.168288e-02 | 
| Age.45_64 |  0.61158346   |  1.50856044   | -0.40636096   |  0.56755764   |  0.74110491   | -0.34422060   | -0.14593886   | -0.265966034  | -0.03774283   | -0.31035566   |  0.07051793   | -0.017563425  | -1.316734e-02 | 
| Age.65+ | -0.13852516   |  1.64372257   | -0.07254647   | -0.80031730   | -0.70377419   | -0.04632698   |  0.15048374   |  0.148413266  |  0.10780013   |  0.27255063   | -0.06456314   |  0.028040098  | -1.551164e-02 | 
| White |  0.15789756   |  0.78689631   |  1.49644450   | -0.04295800   |  0.78025402   |  0.54961654   | -0.28167337   | -0.142506328  | -0.06216483   |  0.23254733   | -0.17613919   |  0.123209851  |  1.569325e-04 | 
| AfricanAmerican | -0.75640297   | -0.04378918   | -1.49342186   |  0.81330169   | -0.35657150   |  0.23355562   | -0.37220576   | -0.294433235  | -0.03276357   |  0.13966165   | -0.32009913   |  0.064859705  |  5.843316e-05 | 
| Latino |  0.33194597   | -0.67736016   | -0.55579679   | -1.62403883   |  0.22605279   |  0.24535658   | -0.03324242   | -0.537233750  |  0.24836671   | -0.13801137   | -0.05304333   | -0.003353578  |  1.607508e-04 | 
| CollegeEducation |  1.84724544   | -0.20310481   | -0.33277386   |  0.06365050   |  0.02168348   |  0.06322172   | -0.38662557   | -0.142603871  | -0.08523715   |  0.38580011   |  0.24210795   | -0.206487580  | -2.685015e-04 | 
| Poverty | -1.77499704   |  0.24274803   | -0.29289044   | -0.43076625   | -0.12136975   |  0.21056820   | -0.41458248   | -0.008377123  | -0.32045016   | -0.02720751   |  0.40445859   |  0.145192351  | -1.764934e-04 | 
| Income.100k+ |  1.69386911   | -0.64663060   | -0.55824364   |  0.20664370   |  0.18140937   | -0.30358564   |  0.26963782   | -0.039151677  |  0.09556263   |  0.18082517   |  0.13465292   |  0.314748069  |  2.371294e-05 | 
| Obesity | -1.62681027   |  0.31456297   | -0.08014995   |  0.64592469   |  0.29817294   |  0.34735075   |  0.53971061   | -0.350115034  |  0.34376927   |  0.20089556   |  0.21225902   | -0.062271989  | -4.148574e-05 | 
| GunOwner | -1.24689958   | -0.60831181   |  0.72181975   | -0.13937141   |  0.01302434   | -1.15933769   | -0.14052115   | -0.359837814  | -0.04174323   |  0.19024904   | -0.05725125   | -0.008496376  |  1.182089e-05 | 
| Unemployment | -0.91788903   | -0.15756035   | -1.11871425   | -0.53060501   |  1.13500278   | -0.18003467   |  0.07491665   |  0.466245963  | -0.08851463   |  0.19114470   | -0.09538173   | -0.049807894  |  2.199158e-05 | 







## Swing States Analysis

In [ ]:
#Restrict to Swing States
data.swing <- df[which(df$SwingState==TRUE),]